# Data preparation

In this notebook we preprocess our data for ML algorithms and create new features.

## Load and reshape data

Import libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime  
from datetime import timedelta  
import gc
import pickle
from sklearn import preprocessing, metrics
import boto3
import pandas as pd
from sagemaker import get_execution_role


Helper functions

In [2]:
# this function is taken from https://www.kaggle.com/ragnar123/very-fst-model
def reduce_mem_usage(df, verbose=True):
    '''Reduce memory usage of dataframe by converting ints and floats 
    Args:
        df: dataframe
            
    Returns:
        dataframe with converted columns
    '''
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


Read data from S3.

In [3]:
role = get_execution_role()
bucket='sagemaker-eu-central-1-594657351600'
prefix = 'kaggle-m5/00_data'


In [4]:
print('Reading files...')
data_location = 's3://{}/{}/{}'.format(bucket, prefix, 'calendar.csv')
calendar = pd.read_csv(data_location)
calendar = reduce_mem_usage(calendar)
print('Calendar has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))

data_location = 's3://{}/{}/{}'.format(bucket, prefix, 'sell_prices.csv')
sell_prices = pd.read_csv(data_location)
sell_prices = reduce_mem_usage(sell_prices)
print('Sell prices has {} rows and {} columns'.format(sell_prices.shape[0], sell_prices.shape[1]))

data_location = 's3://{}/{}/{}'.format(bucket, prefix, 'sales_train_validation.csv')
sales_train_validation = pd.read_csv(data_location)
sales_train_validation = reduce_mem_usage(sales_train_validation)
print('Sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))

data_location = 's3://{}/{}/{}'.format(bucket, prefix, 'sample_submission.csv')
submission = pd.read_csv(data_location)
submission = reduce_mem_usage(submission)
print('Sample submisson has {} rows and {} columns'.format(submission.shape[0], submission.shape[1]))


Reading files...
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Calendar has 1969 rows and 14 columns
Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns
Mem. usage decreased to 95.00 Mb (78.7% reduction)
Sales train validation has 30490 rows and 1919 columns
Mem. usage decreased to  2.09 Mb (84.5% reduction)
Sample submisson has 60980 rows and 29 columns


In [5]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [6]:
sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812


In [7]:
sales_train_validation.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [8]:
submission.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We will work with data in long format, therefore we reshape sales_train_validation. The reshaped variable is called X. This is our feature matrix.

In [9]:
X = pd.melt(sales_train_validation, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'd', value_name = 'demand')
print('Melted sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
X = reduce_mem_usage(X)
X.head()

Melted sales train validation has 30490 rows and 1919 columns
Mem. usage decreased to 3226.27 Mb (0.0% reduction)


,id,item_id,dept_id,cat_id,store_id,state_id,d,demand
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


In [10]:
del sales_train_validation

## Feature engineering

### Remove rows before  release date

We want to determine the start of sales date for every product and remove all rows  from  X before a product release.

Determine  the start of sales date for each item.

In [11]:
start_sales_df = sell_prices.groupby(['store_id','item_id'])['wm_yr_wk'].agg(['min']).reset_index()
start_sales_df = start_sales_df.rename(columns={"min": "wm_yr_wk"})
start_sales_df.head()

,store_id,item_id,wm_yr_wk
0,CA_1,FOODS_1_001,11101
1,CA_1,FOODS_1_002,11101
2,CA_1,FOODS_1_003,11101
3,CA_1,FOODS_1_004,11206
4,CA_1,FOODS_1_005,11101


For each week we calculate the first day of the week.

In [12]:
start_of_week = calendar.groupby(['wm_yr_wk'])['d','date'].agg(['min']).reset_index()
start_of_week.columns = start_of_week.columns.get_level_values(0)
start_of_week = start_of_week.rename(columns={"d": "start_date_d", "date":"start_date"})
start_of_week.head()

,wm_yr_wk,start_date_d,start_date
0,11101,d_1,2011-01-29
1,11102,d_10,2011-02-05
2,11103,d_15,2011-02-12
3,11104,d_22,2011-02-19
4,11105,d_29,2011-02-26


In [13]:
start_sales_df = start_sales_df.merge(start_of_week, on=['wm_yr_wk'], how='left')
del start_of_week
start_sales_df.head()


,store_id,item_id,wm_yr_wk,start_date_d,start_date
0,CA_1,FOODS_1_001,11101,d_1,2011-01-29
1,CA_1,FOODS_1_002,11101,d_1,2011-01-29
2,CA_1,FOODS_1_003,11101,d_1,2011-01-29
3,CA_1,FOODS_1_004,11206,d_400,2012-03-03
4,CA_1,FOODS_1_005,11101,d_1,2011-01-29


Add features from calendar to X based on date.

In [14]:
#X = X.set_index('d').join(calendar[['date', 'd']].set_index('d')).reset_index()
calendar_features = ['date', 'wm_yr_wk', 'weekday', 'month', 'event_name_1', 'event_type_1', 
                      'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'd']
X = X.merge(calendar[calendar_features], on = 'd', how = 'left')

Add start date of sales to X.

In [15]:
X = X.merge(start_sales_df[['store_id', 'item_id', 'start_date']],  on=['store_id', 'item_id'], how='left')


Convert strings to datetime

In [16]:
X['date']= pd.to_datetime(X['date']) 
X['start_date']= pd.to_datetime(X['start_date']) 
calendar['date'] =  pd.to_datetime(calendar['date']) 

Delete rows before start date itemwise

In [17]:
X = X.drop(X[X.date < X.start_date].index)

### Trends

Add the number of days from the begining of sales and from the first date in the dataset.

In [18]:
first_day = pd.Timestamp(X['date'].values.min())
last_day =  pd.Timestamp(X['date'].values.max())

In [19]:
X['days_from_start'] = X['date'] - first_day
X['days_from_start'] = X['days_from_start'].dt.days

In [20]:
X['start_date_from_start'] = X['start_date'] - first_day
X['start_date_from_start'] = X['start_date_from_start'].dt.days

In [21]:
X = X.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')

In [22]:
X.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,wm_yr_wk,...,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,start_date,days_from_start,start_date_from_start,sell_price
0,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12,2011-01-29,11101,...,NaN,NaN,NaN,0,0,0,2011-01-29,0,0,0.459961
1,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2,2011-01-29,11101,...,NaN,NaN,NaN,0,0,0,2011-01-29,0,0,1.559570
2,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,NaN,NaN,NaN,0,0,0,2011-01-29,0,0,3.169922
3,HOBBIES_1_012_CA_1_validation,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,NaN,NaN,NaN,0,0,0,2011-01-29,0,0,5.980469
4,HOBBIES_1_015_CA_1_validation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,d_1,4,2011-01-29,11101,...,NaN,NaN,NaN,0,0,0,2011-01-29,0,0,0.700195


### Sales pattern

In this part we determine forecastability of the time series, as it was done at https://github.com/Mcompetitions/M5-methods

Determine average demand interval.

In [23]:
X['ADI'] = X.groupby(['id'])['demand'].transform(lambda x: len(x)/sum(x > 0))

Compute square of the Coefficient of Variation (CV²).

In [24]:
X['CV2'] = X.groupby(['id'])['demand'].transform(lambda x: (np.std(x[x>0])/np.mean(x[x>0]))**2)

Calculate forecastability.

In [25]:
X['forecastability'] = ((X['ADI'] <= 1.32) & (X['CV2'] < 0.5))*1 + ((X['ADI'] > 1.32) & (X['CV2'] < 0.5))*10 + \
((X['ADI'] <= 1.32) & (X['CV2'] >= 0.5))*100  + ((X['ADI'] > 1.32) & (X['CV2'] > 0.5))*1000 
X['forecastability'] = X['forecastability'].replace({1:'smooth', 10:'intermittent', 100:'erratic', 1000:'lumpy'})

### Aggregations

Add average sales per week per item, per month per item and simply average sales per item. Add maximum items sold.

In [26]:
X['avg_weekday_demand'] = X.groupby(['id', 'weekday'])['demand'].transform('mean') 
X['avg_demand'] =  X.groupby(['id'])['demand'].transform('mean') 
X['max_demand'] =  X.groupby(['id'])['demand'].transform('max') 


Add quantiles

In [27]:
X['quantile025_week'] = X.groupby(['id', 'weekday'])['demand'].transform(lambda x: x.quantile(0.25))

Add maximum of weekday demand.

In [28]:
X['max_weekday_demand'] = X.groupby(['id', 'weekday'])['demand'].transform('max') 

Add average price per item

In [29]:
X['avg_price'] = X.groupby(['id'])['sell_price'].transform('mean') 
X['cheaper_than_usual'] = ( X['sell_price'] < X['avg_price'])

In [30]:
#X = X.drop(columns = ['min_weekday_demand'])

In [31]:
X.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,wm_yr_wk,...,ADI,CV2,forecastability,avg_weekday_demand,avg_demand,max_demand,quantile025_week,max_weekday_demand,avg_price,cheaper_than_usual
0,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12,2011-01-29,11101,...,1.388244,0.865134,lumpy,8.613139,7.229482,91,1.0,47,0.476074,True
1,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2,2011-01-29,11101,...,2.173864,0.787054,lumpy,1.521898,1.186095,20,0.0,10,1.763672,True
2,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,2.086150,0.266013,intermittent,0.799270,0.719289,6,0.0,5,2.982422,False
3,HOBBIES_1_012_CA_1_validation,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,3.198997,0.161255,intermittent,0.492701,0.394145,3,0.0,3,6.468750,True
4,HOBBIES_1_015_CA_1_validation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,d_1,4,2011-01-29,11101,...,1.242208,0.987833,erratic,8.642336,6.064820,83,3.0,52,0.707031,True


### Test set

Reshape test test to long format.

In [32]:
test1_rows = [row for row in submission['id'] if 'validation' in row]
test2_rows = [row for row in submission['id'] if 'evaluation' in row]
test1 = submission[submission['id'].isin(test1_rows)]
test2 = submission[submission['id'].isin(test2_rows)]

In [33]:
test_dates = calendar.loc[1913:1940, 'date']
column_names = test_dates.dt.strftime('%Y-%m-%d').to_list()
column_names.insert(0,'id' )
test1.columns = column_names

In [34]:
test1 = pd.melt(test1, id_vars = ['id'], var_name = 'date')
test1 = test1.drop(columns = 'value')
test1['date'] =  pd.to_datetime(test1['date']) 
test1.head()

,id,date
0,HOBBIES_1_001_CA_1_validation,2016-04-25
1,HOBBIES_1_002_CA_1_validation,2016-04-25
2,HOBBIES_1_003_CA_1_validation,2016-04-25
3,HOBBIES_1_004_CA_1_validation,2016-04-25
4,HOBBIES_1_005_CA_1_validation,2016-04-25


In [35]:
test1 = test1.merge(calendar[calendar_features], on = 'date', how = 'left')
test1.head()

,id,date,wm_yr_wk,weekday,month,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,d
0,HOBBIES_1_001_CA_1_validation,2016-04-25,11613,Monday,4,NaN,NaN,NaN,NaN,0,0,0,d_1914
1,HOBBIES_1_002_CA_1_validation,2016-04-25,11613,Monday,4,NaN,NaN,NaN,NaN,0,0,0,d_1914
2,HOBBIES_1_003_CA_1_validation,2016-04-25,11613,Monday,4,NaN,NaN,NaN,NaN,0,0,0,d_1914
3,HOBBIES_1_004_CA_1_validation,2016-04-25,11613,Monday,4,NaN,NaN,NaN,NaN,0,0,0,d_1914
4,HOBBIES_1_005_CA_1_validation,2016-04-25,11613,Monday,4,NaN,NaN,NaN,NaN,0,0,0,d_1914


In [36]:
X = reduce_mem_usage(X)
gc.collect()

Mem. usage decreased to 6979.41 Mb (20.9% reduction)


1043

In [37]:
X.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'demand', 'date', 'wm_yr_wk', 'weekday', 'month', 'event_name_1',
       'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX',
       'snap_WI', 'start_date', 'days_from_start', 'start_date_from_start',
       'sell_price', 'ADI', 'CV2', 'forecastability', 'avg_weekday_demand',
       'avg_demand', 'max_demand', 'quantile025_week', 'max_weekday_demand',
       'avg_price', 'cheaper_than_usual'],
      dtype='object')

In [38]:
temp = X[['id',  'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'max_demand', 'avg_demand',
          'start_date', 'start_date_from_start', 'avg_price',  'ADI', 'CV2', 'forecastability']].drop_duplicates()
test1 = test1.merge(temp, on = ['id'], how = 'left')
test1['item_id'].unique()

array(['HOBBIES_1_001', 'HOBBIES_1_002', 'HOBBIES_1_003', ...,
       'FOODS_3_825', 'FOODS_3_826', 'FOODS_3_827'], dtype=object)

In [39]:
test1 = test1.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')

In [40]:
test1['days_from_start'] = test1['date'] - first_day
test1['days_from_start'] = test1['days_from_start'].dt.days

In [41]:
temp = X[['id',  'weekday', 'avg_weekday_demand', 'quantile025_week', 'max_weekday_demand']].drop_duplicates()
test1 = test1.merge(temp, on = ['id', 'weekday'], how = 'left')
test1['item_id'].unique()

array(['HOBBIES_1_001', 'HOBBIES_1_002', 'HOBBIES_1_003', ...,
       'FOODS_3_825', 'FOODS_3_826', 'FOODS_3_827'], dtype=object)

In [45]:
test1['cheaper_than_usual'] = ( test1['sell_price'] < test1['avg_price'])
test1['demand'] = 0
X.columns[~X.columns.isin(test1.columns)]

Index([], dtype='object')

In [46]:
test1 = test1[X.columns]

In [47]:
last_X_day = X.date.values.max()

In [48]:
X = pd.concat([X, test1], axis = 0)

In [49]:
events = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
X[events] = X[events].fillna(value='no event')

### Lag features

In [50]:
lags = [28, 29, 35]
lag_cols = [f"lag_{lag}" for lag in lags ]
for lag, lag_col in zip(lags, lag_cols):
    X[lag_col] = X.groupby("id")["demand"].shift(lag)

lags = [28]
windows = [7, 28]
for window in windows:
    for lag,lag_col in zip(lags, lag_cols):
        X[f"rmean_{lag}_{window}"] = X[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(window).mean())

In [51]:
del test1

For each id drop early rows for which lag cannot be calculated

In [53]:
X = X.dropna()

### Categorical features

In [54]:
X = X.drop(columns = ['wm_yr_wk'])

In [55]:
categorical_features = ['id', 'weekday', 'month' ,'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1',
       'event_name_2', 'event_type_2', 'forecastability', 'cheaper_than_usual']

for c in categorical_features:
    col_type = X[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        X[c] = X[c].astype('category')
        

In [56]:
X.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'demand', 'date', 'weekday', 'month', 'event_name_1', 'event_type_1',
       'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI',
       'start_date', 'days_from_start', 'start_date_from_start', 'sell_price',
       'ADI', 'CV2', 'forecastability', 'avg_weekday_demand', 'avg_demand',
       'max_demand', 'quantile025_week', 'max_weekday_demand', 'avg_price',
       'cheaper_than_usual', 'lag_28', 'lag_29', 'lag_35', 'rmean_28_7',
       'rmean_28_28'],
      dtype='object')

In [58]:
X.to_pickle("../01_preprocessed_data/X.pkl")

# From train, validation, and test datasets

Split X into X_train and X_val

In [59]:
n_days = last_day-first_day
n_days = n_days.days
val_size = 0.05
n_val_days = 2*28# round(n_days*val_size)

In [60]:
print("n_val_days {}".format(n_val_days))

n_val_days 56


In [61]:
first_val_day = first_day + timedelta(days = n_days - n_val_days + 1)
first_val_day

Timestamp('2016-02-29 00:00:00')

In [62]:
X_train = X[X['date'] < first_val_day]
y_train = X_train[['demand']]
X_val = X[(X['date'] >= first_val_day) & (X['date'] <= last_X_day)]
y_val = X_val[['demand']]
X_test = X[X['date'] > last_X_day]

Do not include date, start_date and demand as features.

In [71]:
not_features = ['date', 'start_date', 'demand']
numerical_features = X.columns[~X.columns.isin(categorical_features + not_features)]
new_features = X.columns[~X.columns.isin(not_features)]

In [64]:
X_train = X_train.drop(columns = ['demand'])
X_val = X_val.drop(columns = ['demand'])
X_test = X_test.drop(columns = ['demand'])


Form test dataset

In [66]:
X_test = reduce_mem_usage(X_test)
X_train = reduce_mem_usage(X_train)
X_val = reduce_mem_usage(X_val)
X = reduce_mem_usage(X)

Mem. usage decreased to 69.16 Mb (29.8% reduction)
Mem. usage decreased to 3377.07 Mb (30.2% reduction)
Mem. usage decreased to 136.70 Mb (30.0% reduction)
Mem. usage decreased to 3665.99 Mb (33.1% reduction)


Save datasets

In [67]:
X_val[new_features].to_pickle("../01_preprocessed_data/X_val.pkl")
y_val.to_pickle("../01_preprocessed_data/y_val.pkl")
del X_val, y_val

In [68]:
X_test[new_features].to_pickle("../01_preprocessed_data/X_test.pkl")
del X_test

In [69]:
X_train[new_features].to_pickle("../01_preprocessed_data/X_train.pkl")
y_train.to_pickle("../01_preprocessed_data/y_train.pkl")
del X_train, y_train